# Question 1 a)

Running the code below on our dataset we get some insight as to where we got this naive solution of $3145.13, it happens to be the mean value of the order_amount which would be the average of the amount that is typically ordered across all 100 sneaker stores. But there is a problem with this approach:

- There may be outliers that skew the average order value

In [2]:
import pandas as pd

df = pd.read_csv("2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv")

df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


Solution: Let's group the data by the shop_id and then plot the data out to see which stores have an abnormally high order_amount

In [3]:
#Group the shop_id column
grouped_df = df.groupby("shop_id")


#Get the mean order_amount for each shop_id
mean_df = grouped_df.mean()
print(mean_df)


            order_id     user_id  order_amount  total_items
shop_id                                                    
1        2514.568182  833.613636    308.818182     1.954545
2        2299.054545  861.272727    174.327273     1.854545
3        2305.875000  847.708333    305.250000     2.062500
4        2747.607843  866.450980    258.509804     2.019608
5        2498.288889  862.222222    290.311111     2.044444
...              ...         ...           ...          ...
96       2496.176471  865.411765    330.000000     2.156863
97       2508.604167  844.625000    324.000000     2.000000
98       2470.862069  866.793103    245.362069     1.844828
99       2386.000000  846.166667    339.444444     1.740741
100      2356.525000  852.325000    213.675000     1.925000

[100 rows x 4 columns]


In [4]:
import matplotlib.pyplot as plt
plt.plot(mean_df['order_amount'])
plt.show()

<Figure size 640x480 with 1 Axes>

Observing the graph above, we can see that there are two stores that are abnormally high AOV's, let's sort the data frame in order to see which stores this could be

In [5]:
sorted_mean_df = mean_df.sort_values(by=['order_amount'], ascending=False)
print(sorted_mean_df)

            order_id     user_id   order_amount  total_items
shop_id                                                     
42       2441.921569  758.588235  235101.490196   667.901961
78       2663.021739  867.739130   49213.043478     1.913043
50       2162.431818  834.500000     403.545455     2.090909
90       2694.163265  839.857143     403.224490     2.265306
38       2639.714286  837.800000     390.857143     2.057143
...              ...         ...            ...          ...
53       2393.411765  858.544118     214.117647     1.911765
100      2356.525000  852.325000     213.675000     1.925000
32       2299.071429  856.809524     189.976190     1.880952
2        2299.054545  861.272727     174.327273     1.854545
92       2449.142857  865.095238     162.857143     1.809524

[100 rows x 4 columns]


There seems to be 2 Shopify Stores that are causing an issue in our AOV calculation and skewing the data, shop 42 and shop 78, let's go back to our original data set and now find the AOV without including these stores by filtering them in order to get a better estimate of the average order value.

In [6]:
#Filter out shop 42 and 78
filtered_df = df.query("shop_id != '42' \
                       and shop_id != '78'")

filtered_df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,4903.000000,4903.000000,4903.000000,4903.000000,4903.000000
mean,2499.584540,49.900877,849.858862,300.155823,1.995717
std,1444.221163,29.154367,86.887947,155.941112,0.982602
min,1.000000,1.000000,700.000000,90.000000,1.000000
25%,1246.500000,24.000000,776.000000,163.000000,1.000000
50%,2499.000000,50.000000,850.000000,284.000000,2.000000
75%,3750.500000,74.000000,925.000000,386.500000,3.000000
max,5000.000000,100.000000,999.000000,1086.000000,8.000000


This gives us a much more realistic AOV of $300.16 rounded to the nearest cent, which is inline with shops that sell sneakers

# Question 1 b)

I would report three metrics for this dataset, the AOV with the noisy data filtered out, the median order value and the modal order value (i.e the mode), this will give us a much more reasonable overview of the dataset. If we want to increase the value of our orders then we need the metrics that measure our data's central tendency (mean, median and mode) however no measure of central tendency is best, but using only one is certainly the worst hence why we should report three metrics. 

# Question 1 c)

- AOV with outliers filtered (mean) = $300.16

- Median order value (analyzed below) = $284.00

- Modal order value (analyzed below) = $153.00

In [19]:
#Sort the dataframe in order to find the median
sorted_df = df.sort_values(by=['order_amount'])

sorted_df.median()['order_amount']

284.0

In [20]:
df.mode()['order_amount'][0]

153.0

# Question 2 a)

```sql

SELECT COUNT(ShipperID)
FROM Orders
WHERE ShipperID == 1;

```

The number of orders shipped by Speedy Express is 54

# Question 2 b)

```sql
SELECT Employees.LastName 
FROM Orders 
INNER JOIN Employees ON Orders.EmployeeID=Employees.EmployeeID 
GROUP BY Employees.LastName 
ORDER BY Count(Orders.OrderID) DESC LIMIT 1;

```

The last name of the employee with the most orders is Peacock

# Question 2 c)

Slightly ambigious question in terms of the wording, are we trying to ask which product had the highest quantity ordered or the highest amount of orders placed for a single product?




If we are trying to find the product with the highest quantity ordered then the following will work

```sql
SELECT product.ProductName, SUM(Quantity) AS QuantityOrdered
FROM Orders AS orders, OrderDetails AS order_details, Customers AS customer, Products AS product
WHERE customer.Country == "Germany" AND order_details.OrderID == orders.OrderID AND order_details.ProductID == product.ProductID AND customer.CustomerID == orders.CustomerID
GROUP BY product.ProductID
ORDER BY QuantityOrdered DESC
LIMIT 1;
```
In this case Boston Crab Meat is the highest Quantity Ordered at 160

If we are trying to find the product with the highest amount of orders placed the following will work

```sql
SELECT product.ProductName, COUNT(Quantity) AS NumberOfOrders
FROM Orders AS orders, OrderDetails AS order_details, Customers AS customer, Products AS product
WHERE customer.Country == "Germany" AND order_details.OrderID == orders.OrderID AND order_details.ProductID == product.ProductID AND customer.CustomerID == orders.CustomerID
GROUP BY product.ProductID
ORDER BY NumberOfOrders DESC
LIMIT 1;
```
In this case Gorgonzola Telino is the product that has the most orders placed on it at 5, regardless of quanitiy ordered for each order


